# *Movie Recommend System*

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
movies = pd.read_csv(r"C:\Users\SHIVAM GHUGE\Downloads\tmdb_5000_movies.csv",encoding='latin1')
credits = pd.read_csv(r"C:\Users\SHIVAM GHUGE\Downloads\tmdb_5000_credits.csv",encoding='latin1')

# since the datset is not encoded in utf-8 so we have to specify it seperately

In [3]:
credits.head()

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [4]:
movies = movies.merge(credits,how='inner',on='title')

In [5]:
movies.isnull().sum()

budget                     0
genres                     0
homepage                3096
id                         0
keywords                   0
original_language          0
original_title             0
overview                   3
popularity                 0
production_companies       0
production_countries       0
release_date               1
revenue                    0
runtime                    2
spoken_languages           0
status                     0
tagline                  844
title                      0
vote_average               0
vote_count                 0
movie_id                   0
cast                       0
crew                       0
dtype: int64

***formatting movie genres***

In [6]:
# some features are in JSON format we have to change them.We want generes and keyword as it is most important to form relation between movies.

movies['genres'].loc[0]


'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [7]:
import json

x = json.loads(movies['genres'][0])
x

[{'id': 28, 'name': 'Action'},
 {'id': 12, 'name': 'Adventure'},
 {'id': 14, 'name': 'Fantasy'},
 {'id': 878, 'name': 'Science Fiction'}]

In [8]:
for i in x:
  print(i['name'])

Action
Adventure
Fantasy
Science Fiction


In [9]:
' '.join(''.join(ele['name'].split()) for ele in x)

# we created string of generes,we do this for each row
# same process is done with geners column

'Action Adventure Fantasy ScienceFiction'

In [10]:
def generes_and_keywords(row):
  genre = json.loads(row['genres'])
  genres = ' '.join(''.join(gen['name'].split()) for gen in genre)

  keyword = json.loads(row['keywords'])
  keywords = ' '.join(''.join(key['name'].split()) for key in keyword)

  return f"{genres} {keywords}"

In [11]:
movies.apply(generes_and_keywords,axis=1)[0]

'Action Adventure Fantasy ScienceFiction cultureclash future spacewar spacecolony society spacetravel futuristic romance space alien tribe alienplanet cgi marine soldier battle loveaffair antiwar powerrelations mindandsoul 3d'

In [12]:
movies['types'] = movies.apply(generes_and_keywords,axis=1)

# these are used to vectorize using TF-IDF,we then compare our movie title with these vectors and create similarity between them.

In [13]:
movies['types'][0]

'Action Adventure Fantasy ScienceFiction cultureclash future spacewar spacecolony society spacetravel futuristic romance space alien tribe alienplanet cgi marine soldier battle loveaffair antiwar powerrelations mindandsoul 3d'

***movie genres***

In [14]:
def genres(row):
  List = []
  genre = json.loads(row['genres'])
  for i in genre:
    List.append(i['name'])

  return List

In [15]:
movies['genres'] = movies.apply(genres,axis=1)

***movie cast***

In [16]:
def cast(row):
  casting = []
  obj = json.loads(row['cast'])
  for i in obj:
    casting.append(i['name'])

  return casting

In [17]:
movies['cast'] = movies.apply(cast,axis=1)

***Director of movie***

In [18]:
def director(row):
  obj = json.loads(row['crew'])
  for i in obj:
    if i['job'] == 'Director':
      return i['name']

In [19]:
movies['director'] = movies.apply(director,axis=1)

In [20]:
movies.columns

Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count', 'movie_id', 'cast', 'crew', 'types', 'director'],
      dtype='object')

In [21]:
movies_processed = movies[['title','genres','types','cast','director']]

In [22]:
movies_processed.head()

,title,genres,types,cast,director
0,Avatar,"[Action, Adventure, Fantasy, Science Fiction]",Action Adventure Fantasy ScienceFiction cultur...,"[Sam Worthington, Zoe Saldana, Sigourney Weave...",James Cameron
1,Pirates of the Caribbean: At World's End,"[Adventure, Fantasy, Action]",Adventure Fantasy Action ocean drugabuse exoti...,"[Johnny Depp, Orlando Bloom, Keira Knightley, ...",Gore Verbinski
2,Spectre,"[Action, Adventure, Crime]",Action Adventure Crime spy basedonnovel secret...,"[Daniel Craig, Christoph Waltz, Léa Seydoux, R...",Sam Mendes
3,The Dark Knight Rises,"[Action, Crime, Drama, Thriller]",Action Crime Drama Thriller dccomics crimefigh...,"[Christian Bale, Michael Caine, Gary Oldman, A...",Christopher Nolan
4,John Carter,"[Action, Adventure, Science Fiction]",Action Adventure ScienceFiction basedonnovel m...,"[Taylor Kitsch, Lynn Collins, Samantha Morton,...",Andrew Stanton


***Vectorization***

In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(max_features=2000)

In [24]:
vectors = tfidf.fit_transform(movies_processed['types'])

In [25]:
vectors

<4809x2000 sparse matrix of type '<class 'numpy.float64'>'
	with 37336 stored elements in Compressed Sparse Row format>

*creating word to index and index to word mapping for title of movies*

In [26]:
title = movies_processed['title']

In [27]:
title

0                                         Avatar
1       Pirates of the Caribbean: At World's End
2                                        Spectre
3                          The Dark Knight Rises
4                                    John Carter
                          ...                   
4804                                 El Mariachi
4805                                   Newlyweds
4806                   Signed, Sealed, Delivered
4807                            Shanghai Calling
4808                           My Date with Drew
Name: title, Length: 4809, dtype: object

In [28]:
word2indx = {}
index = 0

for i in title:
  word2indx[i] = index
  index += 1

In [ ]:
word2indx

***generating recommendations***

In [30]:
idx = word2indx['Avatar']

In [31]:
# Dark Water movie corresponding row in our tfidf matrix

movie = vectors[idx]

In [32]:
movie.shape

(1, 2000)

In [33]:
vectors.shape

(4809, 2000)

***cosine_similarity***

*Cosine similarity measures the similarity between two vectors of an inner product space. It is measured by the cosine of the angle between two vectors and determines whether two vectors are pointing in roughly the same direction.*

*Cosine similarity is a value bound by a constrained range of 0 and 1.Suppose the angle between the two vectors were 90 degrees. In that case, the cosine similarity will have a value of 0 i.e., no similarity*

In [34]:
from sklearn.metrics.pairwise import cosine_similarity

In [35]:
help(cosine_similarity)

Help on function cosine_similarity in module sklearn.metrics.pairwise:

cosine_similarity(X, Y=None, dense_output=True)
    Compute cosine similarity between samples in X and Y.
    
    Cosine similarity, or the cosine kernel, computes similarity as the
    normalized dot product of X and Y:
    
        K(X, Y) = <X, Y> / (||X||*||Y||)
    
    On L2-normalized data, this function is equivalent to linear_kernel.
    
    Read more in the :ref:`User Guide <cosine_similarity>`.
    
    Parameters
    ----------
    X : {array-like, sparse matrix} of shape (n_samples_X, n_features)
        Input data.
    
    Y : {array-like, sparse matrix} of shape (n_samples_Y, n_features),             default=None
        Input data. If ``None``, the output will be the pairwise
        similarities between all samples in ``X``.
    
    dense_output : bool, default=True
        Whether to return dense output even when the input is sparse. If
        ``False``, the output is sparse if both input arr

In [36]:
similarity = cosine_similarity(movie,vectors)

In [37]:
similarity.shape

(1, 4809)

In [38]:
scores = similarity.flatten()

In [39]:
scores

array([1.        , 0.03452781, 0.03690872, ..., 0.01661789, 0.        ,
       0.        ])

In [40]:
recommend = scores.argsort()[::-1][:6]

# these index has top 5 similarity score for our movie

In [41]:
recommend

array([   0,   47, 3219, 1290, 3737,   61], dtype=int64)

In [42]:
# converting these index back to similar movies names

movies['title'].iloc[recommend]


0                        Avatar
47      Star Trek Into Darkness
3219                 Barbarella
1290         A Monster in Paris
3737                      Cargo
61            Jupiter Ascending
Name: title, dtype: object

*function to recommend any movie related to our search*

In [43]:
def recommender(movie_name):
  index = word2indx[movie_name]

  movie_tfidf = vectors[index]

  similar = cosine_similarity(movie_tfidf,vectors)

  score = similar.flatten()

  top_5_index = score.argsort()[::-1][:6]

  top_5_movies = movies['title'].iloc[top_5_index]

  return top_5_movies

In [44]:
recommender('The Dark Knight Rises')

3       The Dark Knight Rises
65            The Dark Knight
119             Batman Begins
2870                Arbitrage
210            Batman & Robin
1746               Kick-Ass 2
Name: title, dtype: object

In [45]:
def filter_movies(search,num_results=10):
  genres = movies_processed[movies_processed['genres'].apply(lambda x: search in x and 'Adventure' not in x and 'Fantasy' not in x)]
  if not genres.empty:
    if len(genres['genres']) < num_results:
      return genres['title'][:len(genres['genres'])]
    return genres['title'][:num_results]

  cast = movies_processed[movies_processed['cast'].apply(lambda x: search in x)]
  if not cast.empty:
    if len(cast['cast']) < num_results:
      return cast['title'][:len(cast['cast'])]
    return cast['title'][:num_results]

  director = movies_processed[movies_processed['director'] == 'search']
  if not director.empty:
    return director['title']

  return 'No Results!'

In [46]:
import pickle

with open('word_index','wb') as handle:
    pickle.dump(word2indx,handle)

In [47]:
with open('movie_vectors','wb') as handle:
    pickle.dump(vectors,handle)

In [50]:
movies_processed.to_csv('Movies.csv',index=False)

# ***Using LSA for recommendation***

In [ ]:
movies['genres'][0]

'Action Adventure Fantasy ScienceFiction cultureclash future spacewar spacecolony society spacetravel futuristic romance space alien tribe alienplanet cgi marine soldier battle loveaffair antiwar powerrelations mindandsoul 3d'

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer

# Split the genres and transform into a binary matrix
mlb = MultiLabelBinarizer()
genre_matrix = mlb.fit_transform(movies['genres'].str.split('\n'))

In [ ]:
genre_matrix

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

***modeling SDA***

In [ ]:
from sklearn.decomposition import TruncatedSVD

In [ ]:
svd = TruncatedSVD(n_components=10)

In [ ]:
X = vectors                         # using tfidf matrix

In [ ]:
svd.fit(X)

TruncatedSVD(n_components=10)

In [ ]:
Z = svd.transform(X)

In [ ]:
Z.shape

(4809, 10)

*creating similarities among movies*

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
similarity = cosine_similarity(Z)

In [ ]:
similarity.shape

(4809, 4809)

In [ ]:
movie_idx = word2indx['The Dark Knight']
movie_scores = list(enumerate(similarity[movie_idx]))

In [ ]:
movie_scores = sorted(movie_scores,key=lambda x: x[1], reverse=True)           # key is specified to sort on the basis of score not the index

In [ ]:
top_5 = movie_scores[:6]

In [ ]:
for i,j in top_5:
  print(movie_titles[i])

The Dark Knight
The Dark Knight Rises
The Proposition
The Glimmer Man
Hitman
xXx: State of the Union
